# Normative matching problem: a proof algorithm

In [1]:
import itertools # sert pour la fonction permutations
import math # sert pour ceil et floor
from score_functions import *
from display_functions import *

In [2]:
class Allocation:
    
    """

    Une Allocation est intuitivement une liste qui indique à quel élève chaque cours est alloué.
    On a donc besoin en input d'un uplet. 
    Par exemple, ('A', 'C', 'B'), pour 3 élèves, signifie que le cours A est alouée à l'élève 1, C à 2, B à 3.
    Mais on veut également (pour pouvoir comparer les Allocation entre elles selon la satisfaction
    qu'elles donnent aux élèves) connaître la satisfaction associée à chaque Allocation.
    Pour ce faire, on a besoin d'avoir l'information des préférences de chaque élève : c'est pourquoi
    Allocation prend également en input un objet ProfilDePrefs, auquel elle est rattachée
    via son attribut profil_de_prefs, et qui permet de calculer l'attribut satisfaction.

    Une Allocation est ainsi caractérisée par 3 attributs : un uplet qui indique les cours affectés,
    un profil_de_prefs et une satisfaction.
    Les attributs uplet et profil_de_prefs sont donnés d'emblée ; l'attribut satisfaction est calculé
    à partir des 2 premiers via la méthode satisfaction().
    La méthode satisfaction_etudiant() ne sert qu'à la méthode satisfaction()

    
    ### Attributs ###
    
    - uplet
        - ex : ('A', 'B', 'C')
    - profil_de_prefs
        - objet de classe ProfilDePrefs
    - satisfaction
        - satisfaction ordonnée (par ordre décroissant)
        - ex : [4, 3, 3, 1]
    
    
    ### Méthodes ###
    
    - satisfaction_etudiant(i_etudiant)
        - arguments
            - i_etudiant
                - indice de l'étudiant dont on veut la satisfaction
        - return
            - satisfaction de l'étudiant
            - type : int
    
    - satisfaction()
        - return
            - satisfaction de l'allocation
            - type : tuple
            - ex : (4, 3, 3, 1)
          
        
    """
    
    def __init__(self, profil_de_prefs_, allocation_uplet_):
        
        self.uplet = allocation_uplet_
        # uplet est de la forme ('A', 'C', 'B')
        
        self.profil_de_prefs = profil_de_prefs_
        # profil_de_prefs est un objet de classe profil_de_prefs
        
        self.satisfaction = self.satisfaction()
        # satisfaction est de la forme [4, 3, 3, 1] (satisfactions des étudiants classées dans l'ordre décroissant)

        self.score_1 = profil_de_prefs_.score_1_function(self.satisfaction)
        # score_1 de l'Allocation ; ex : 14 pour score_utilitarisme, 13445 pour score_leximin

        self.score_2 = profil_de_prefs_.score_2_function(self.satisfaction)
        # score_2 de l'Allocation
                        
    def satisfaction_etudiant(self, i_etudiant_):
        """
        Méthode dont la seule utilité est de calculer self.satisfaction
        (elle n'est d'ailleurs utilisée que dans la méthode self.satisfaction())
        """
        satisfaction_lineaire = len(self.uplet) - self.profil_de_prefs.uplet[i_etudiant_].index(self.uplet[i_etudiant_])
        # on pourrait imaginer d'autres types de transformations non linéaires !
        return satisfaction_lineaire
        
        # renvoie la satisfaction d'un étudiant en fonction de son indice (en utilisant l'information du profil_de_prefs
        # de l'allocation)

    def satisfaction(self):
        satisfaction_lst = []
        for i_etudiant in range(len(self.uplet)):
            satisfaction_lst.append(self.satisfaction_etudiant(i_etudiant))
        satisfaction_lst.sort(reverse=True) # range par ordre décroissant
        return tuple(satisfaction_lst)
        # renvoie la satisfaction de l'allocation, qui a la forme (4, 3, 3, 1)


class ProfilDePrefs:
    
    """

    Un ProfilDePrefs (pour "profil de préférences") est intuitivement une liste de préférences
    pour chaque élève.
    On a donc besoin en input  d'un uplet.
    Par exemple, [('A', 'B', 'C'), ('B', 'A', 'C'), ('A', 'C', 'B')] pour 3 élèves, indique que
        - l'élève 1 préfère A, puis B, puis C ;
        - l'élève 2 préfère B, puis A, puis C ;
        - l'élève 3 préfère A, puis C, puis B.
    Un profil de préférences a l'interprétation mathématique suivante :
    il s'agit d'une combinaison avec remise, au sens mathématique du terme, de préférences, entendues comme des listes
    ordonnées de cours (ex : ('A', 'C', 'B')) ; plus précisément,
        - le sens n'importe pas, car on suppose que les élèves se valent tous (symétrie des élèves). Ainsi,
            - (ABC, ABC, CBA) et
            - (CBA, ABC, ABC)
          représentent un même ProfilDePrefs (peu nous importe quel élève à quel préférence,
          à la fin on aura les mêmes possibilités de les satisfaire)
        - le "avec remise" importe : plusieurs élèves peuvent avoir les mêmes préférences !
    
    Ce qui nous intéresse, c'est de connaître les allocations normativement efficaces.
    Pour cela, on a besoin de savoir selon quelles conceptions normatives comparer les allocations :
    c'est pourquoi on prend en input 2 fonctions de score (qui donnent un score à chaque allocation,
    pour pouvoir ensuite les comparer selon ce score).
    
    On détermine, à la lumière des fonctions de score qui permettent une comparaison normative des allocations,
    les allocations normativement efficaces.
    
    ### Attributs ###
    
    - uplet
        - ex : [('A', 'B', 'C'), ('B', 'A', 'C'), ('A', 'C', 'B')]
        - type : liste de tuples
    - score_1_function
        - type : fonction
    - score_2_function
        - type : fonction
    - allocations
        - liste des allocations possibles
        - type : list d'objets Allocation
    - allocations_efficaces
        - liste des allocations qui créent un dilemme
        - type : list d'objets Allocation
    
        
    ### Méthodes ###

    - allocations_efficaces()
        - return
            - liste des allocations qui créent un dilemme
            - type : list d'objets Allocation
    
    """

    def __init__(self, profil_de_prefs_uplet_, score_1_function_, score_2_function_):
        
        self.uplet = profil_de_prefs_uplet_
        # uplet est de la forme [('A', 'B', 'C'), ('B', 'A', 'C'), ('A', 'C', 'B')]

        self.score_1_function = score_1_function_
        self.score_2_function = score_2_function_


        self.allocations = []
        
        # for allocation_uplet in permutations:
        # permutations : [('A', 'B', 'C'), ('A', 'C', 'B'), ('B', 'A', 'C'),
        #                 ('B', 'C', 'A'), ('C', 'A', 'B'), ('C', 'B', 'A')]
        for allocation_uplet in list(itertools.permutations(sorted(self.uplet[0]))):
            # pour ne pas être dépendant de l'objet permutations, que l'on ne définit plus globalement
            allocation = Allocation(self, allocation_uplet)
            self.allocations.append(allocation)

                
        self.allocations_efficaces = self.allocations_efficaces_()
        
    
    def allocations_efficaces_(self):
        
        allocations_efficaces_dict = {"allocations": [],
                                     "sorted_scores_1": [],
                                     "sorted_scores_2": []}

        satisfactions_deja_vues = set()
        
        for allocation in self.allocations:
            satisfaction = allocation.satisfaction

            if satisfaction not in satisfactions_deja_vues:

                length = len(allocations_efficaces_dict["allocations"])

                indice_u, indice_v = length + 0.5, length + 0.5

                for i in range(length):
                    if allocations_efficaces_dict["sorted_scores_1"][i] < allocation.score_1:
                        indice_u = i + 0.5
                        break
                    elif allocations_efficaces_dict["sorted_scores_1"][i] == allocation.score_1:
                        indice_u = i + 1
                        break
                    
                for j in range(length):
                    if allocations_efficaces_dict["sorted_scores_2"][j] > allocation.score_2:
                        indice_v = j + 0.5
                        break
                    elif allocations_efficaces_dict["sorted_scores_2"][j] == allocation.score_2:
                        indice_v = j + 1
                        break
                
                if indice_u < indice_v:
                    lim_inf, lim_sup = math.ceil(indice_u), math.floor(indice_v)

                    allocations_efficaces_dict["sorted_scores_1"] = allocations_efficaces_dict["sorted_scores_1"][:lim_inf-1] + \
                        allocations_efficaces_dict["sorted_scores_1"][lim_sup:]
                    allocations_efficaces_dict["sorted_scores_2"] = allocations_efficaces_dict["sorted_scores_2"][:lim_inf-1] + \
                        allocations_efficaces_dict["sorted_scores_2"][lim_sup:]
                    allocations_efficaces_dict["allocations"] = allocations_efficaces_dict["allocations"][:lim_inf-1] + \
                        allocations_efficaces_dict["allocations"][lim_sup:]

                    allocations_efficaces_dict["sorted_scores_1"].insert(lim_inf-1, allocation.score_1)
                    allocations_efficaces_dict["sorted_scores_2"].insert(lim_inf-1, allocation.score_2)
                    allocations_efficaces_dict["allocations"].insert(lim_inf-1, allocation)

                elif indice_u == indice_v:
                    lim_inf, lim_sup = math.ceil(indice_u), math.floor(indice_v)

                    allocations_efficaces_dict["sorted_scores_1"].insert(lim_inf-1, allocation.score_1)
                    allocations_efficaces_dict["sorted_scores_2"].insert(lim_inf-1, allocation.score_2)
                    allocations_efficaces_dict["allocations"].insert(lim_inf-1, allocation)

                satisfactions_deja_vues.add(satisfaction)

                        
        return allocations_efficaces_dict["allocations"]


In [3]:
# i permet de savoir combien de permutations ont déjà été ajoutées dans la liste i_uplet (on pourrait directement utiliser len(i_uplet))
# perm_id_min permet d'ajouter des permutations uniquement par ordre croissant d'index
def generation(n, P_uplet, perm_id_min, permutations, score_1_function, score_2_function, uniqueness, unique_cases):
    if len(P_uplet) == n:
        P = ProfilDePrefs(P_uplet, score_1_function, score_2_function)
        if len(P.allocations_efficaces) > 1:
            # seuls les cas où il y a dilemme nous intéressent,
            # i.e. les cas où  il y a au moins 2 allocations efficaces

            if uniqueness:

                ### Vérification que le dilemme est inédit

                satisfactions_lst = []
                # cette liste stocke les satisfactions dilemme !

                for allocation in P.allocations_efficaces:
                    satisfactions_lst.append(allocation.satisfaction)

                if satisfactions_lst not in unique_cases:
                # on vérifie qu'on a pas déjà une ProfilDePrefs
                # qui a fait popper le même dilemme au niveau des
                # satisfactions, parce qu'on ne veut pas de dilemme doublon

                    unique_cases.append(satisfactions_lst.copy())

                    print_allocations_lst(P.allocations_efficaces)
            
            else:
                print_allocations_lst(P.allocations_efficaces)
    else:
        for i in range(perm_id_min, math.factorial(n)):
            generation(n, P_uplet + [permutations[i]], i, permutations, score_1_function, score_2_function, uniqueness, unique_cases)


def print_dilemmes(n, score_1_function, score_2_function, uniqueness=True):
    permutations = list(itertools.permutations(["A", "B", "C", "D", "E", "F"][0:n]))
    # on commence en ajoutant directement la première permutation, car par renommage des variables,
    # tout profil de préférences est analogue à un profil dont les préférences de la premières personne
    # sont A > B > C > ...
    generation(n, [permutations[0]], 0, permutations, score_1_function, score_2_function, uniqueness, [])


In [4]:
print_dilemmes(4, score_utilitarisme, score_leximin, uniqueness=True)

Profil de préférences

[['A' 'B' 'C' 'D']
 ['A' 'B' 'C' 'D']
 ['A' 'B' 'C' 'D']
 ['C' 'A' 'D' 'B']]

2
   Satisfaction  Score_1  Score_2    Allocation
0  (4, 4, 3, 1)       12     1344  (A, B, D, C)
1  (4, 3, 2, 2)       11     2234  (A, B, C, D)


Profil de préférences

[['A' 'B' 'C' 'D']
 ['A' 'B' 'C' 'D']
 ['B' 'A' 'C' 'D']
 ['C' 'A' 'D' 'B']]

2
   Satisfaction  Score_1  Score_2    Allocation
0  (4, 4, 4, 1)       13     1444  (A, D, B, C)
1  (4, 4, 2, 2)       12     2244  (A, C, B, D)


Profil de préférences

[['A' 'B' 'C' 'D']
 ['A' 'B' 'C' 'D']
 ['B' 'C' 'A' 'D']
 ['C' 'A' 'D' 'B']]

2
   Satisfaction  Score_1  Score_2    Allocation
0  (4, 4, 4, 1)       13     1444  (A, D, B, C)
1  (4, 3, 3, 2)       12     2334  (A, B, C, D)


Profil de préférences

[['A' 'B' 'C' 'D']
 ['A' 'B' 'C' 'D']
 ['B' 'D' 'A' 'C']
 ['D' 'C' 'A' 'B']]

2
   Satisfaction  Score_1  Score_2    Allocation
0  (4, 4, 4, 2)       14     2444  (A, C, B, D)
1  (4, 3, 3, 3)       13     3334  (A, B, D, C)


